In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')

# import embedding methods
from embedding.bernoulli import Bernoulli
from embedding.KL import KL
from embedding.matrix_factorization import MatrixFactorization

# import evaluation methods
from evaluation import evaluate_link_prediction
from evaluation import evaluate_node_classification

# import utils
from utils import graph_util
from utils import plot_util
from utils import model_util

# visualization
%matplotlib inline
#sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

## Experiment 1

In [2]:
# Experiment 1

exp = {
    "max_epochs": 1000,
    "learning_rate": 1e-2, #Adam
    "weight_decay": 1e-7,
    
    "link_prediction": True,
    "link_pred_num_rounds": 1,
    "link_pred_train_ratio": 0.8,
    "link_pred_eval_every_n_steps": 25,
    "link_pred_edge_emb_method": "average",
    
    "node_classification": True,
    "node_class_num_rounds": 1,
    "node_class_train_ratio": 0.8,
    "node_class_eval_every_n_steps": 25,
}

# pick datasets
datasets = ["cora", "citeseer", "polblogs"] # parliament, hvr, purdue_facebook

model_1 = MatrixFactorization(embedding_dimension=64, matrix_type="adjacency")
model_2 = MatrixFactorization(embedding_dimension=64, matrix_type="unnormalized_lapla")
model_3 = Bernoulli(embedding_dimension=64, distance_meassure='sigmoid')
model_4 = KL(embedding_dimension=64, score_type="basic")


embedding_methods = [model_1, model_2, model_3, model_4]

# setup folders to store experiment setup summary and results
result_folder = plot_util.setup_folders_and_summary_files(exp, datasets, embedding_methods)
print(f'The results of the current experiment are stored at experiments/{result_folder}')

for dataset in datasets:
    
    # load dataset
    A, y = graph_util.load_dataset(dataset)
    
    
    for model in embedding_methods:
        
        # do link prediction
        if(exp["link_prediction"]):
            link_prediction_folder = result_folder + "/link_prediction"
            evaluate_link_prediction.expLP(A,dataset,model,exp["link_pred_num_rounds"],
                                           link_prediction_folder, train_ratio=exp["link_pred_train_ratio"], 
                                           edge_emb_method=exp["link_pred_edge_emb_method"],train_epochs=exp["max_epochs"],
                                           eval_epochs=exp["link_pred_eval_every_n_steps"], undirected=True)

        # do node classification
        if(exp["node_classification"]):
            node_classification_folder = result_folder + "/node_classification"
            evaluate_node_classification.expNC(A,y,dataset,model,exp["node_class_num_rounds"],
                                               node_classification_folder, train_ratio=exp["node_class_train_ratio"],
                                               train_epochs=exp["max_epochs"],eval_epochs=exp["node_class_eval_every_n_steps"],undirected=True)

The results of the current experiment are stored at experiments/results/2019_12_08_15_23
	Link prediction evaluation has started...
	Node classification evaluation has started...
	Link prediction evaluation has started...
	Node classification evaluation has started...
	Link prediction evaluation has started...


AttributeError: 'Bernoulli' object has no attribute '_adj'